## Part 1. Make a connection

In [55]:
import sqlite3
from sqlite3 import Error
import pandas as pd

In [56]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    
    return conn

In [57]:
pathToFile = "../data/"
fileName = "HongKong_Air.db"
dbconnection = create_connection(pathToFile + fileName)

## Part 2. Define Function

### List the selection of date and district

In [58]:
# list all the date of the month
def ALLDate(dbconnection):
    print("Please choose one day:")
    with dbconnection:
        sql = "SELECT DISTINCT Date FROM AQHI"
        cur = dbconnection.cursor()
        cur.execute(sql)
        date_result = cur.fetchall()
        for result in date_result:
            print(result[0])

In [59]:
# list the districts in Hong Kong
def ALLDistrict(dbconnection):
    print("Please choose one district:")
    with dbconnection:
        sql = "SELECT DISTINCT District FROM AQHI"
        cur = dbconnection.cursor()
        cur.execute(sql)
        district_result = cur.fetchall()
        for result in district_result:
            print(result[0])

### Define the query sentence function

In [60]:
# sentence 1: The districts with good air quality during the day

def AQHI(Date, dbconnection):
    with dbconnection:
        sql = "SELECT District FROM AQHI WHERE Date = ? and AQHI <= 2 "
        cur = dbconnection.cursor()
        cur.execute(sql, [Date])
        result = cur.fetchall()
        if len(result) == 0: # Determines whether the list is empty
            print("No districts in Hong Kong with good air quality")
        else:
            print('In{',Date,'}Districts with good air quality is(are):')
            for tresult in result:
                print(tresult[0])
            
def selectday(dbconnection):
    ALLDate(dbconnection)
    block = input("Please input the date:")
    block = block.strip()
    AQHI(block, dbconnection)

In [61]:
# sentence 2:
def BadAQHI(District, dbconnection):
    with dbconnection:
        sql = "SELECT Date FROM AQHI WHERE District = ? and AQHI >= 4 "
        cur = dbconnection.cursor()
        cur.execute(sql, [District])
        result = cur.fetchone()
        tresult = result[0]
        if len(result) == 0: # Determines whether the list is empty
            print('The air quality in',District,'is generally good,no bad air quality day')
        else:
            print('In{',District,'}the worst day for air quality is{',tresult,'}')
            
def selectdistrict(dbconnection):
    ALLDistrict(dbconnection)
    block2 = input("Please input your district:")
    block2 = block2.strip()
    BadAQHI(block2, dbconnection)

In [62]:
# sentence 3
def AvgAQHI(District, dbconnection):
    with dbconnection:
        sql = "SELECT AQHI FROM AQHI GROUP BY District HAVING AVG(AQHI) and District =? "
        cur = dbconnection.cursor()
        cur.execute(sql, [District])
        Avgresult = cur.fetchone()
        tAvgresult = Avgresult[0]
        print('In{',District,'}The average air quality is:{',tAvgresult,'}')
        
def selectdistrictAvg(dbconnection):
    ALLDistrict(dbconnection)
    block3 = input("Please input your district:")
    block3 = block3.strip()
    AvgAQHI(block3, dbconnection)

In [63]:
# sentence 4
def mainpollutant(District, dbconnection):
    with dbconnection:
        sql = "SELECT O3, NOX, RSP FROM pollutant WHERE District =? "
        cur = dbconnection.cursor()
        cur.execute(sql, [District])
        pollutant_result = cur.fetchone()
        print('The main pollutants index of{', District,'}are:')
        main = ["O3", "NOX", "RSP"]
        for i in range(0, 3):
            print(main[i], ":", pollutant_result[i])
                
def pollinDistrict(Date, dbconnection):
    ALLDistrict(dbconnection)
    with dbconnection:
        sql = "SELECT O3, NOX, RSP FROM pollutant WHERE Date =? "
        cur = dbconnection.cursor()
        cur.execute(sql, [Date])
        result = cur.fetchone()
        block5 = input('your district:')
        block5 = block5.strip()
        mainpollutant(block5, dbconnection)
            
def selectmain_date(dbconnection):
    ALLDate(dbconnection)
    block4 = input("Please input your date:")
    block4 = block4.strip()
    pollinDistrict(block4, dbconnection)

In [64]:
# sentence 5
def AQHI_index(District, dbconnection):
    with dbconnection:
        sql = "SELECT AQHI FROM AQHI WHERE District =? "
        cur = dbconnection.cursor()
        cur.execute(sql, [District])
        AQHIresult = cur.fetchone()
        tAQHIresult = AQHIresult[0]
        print('In{',District,'}, the air quality index is:{',tAQHIresult,'}')
                
def AQHI_District(Date, dbconnection):
    ALLDistrict(dbconnection)
    with dbconnection:
        sql = "SELECT AQHI FROM AQHI WHERE Date =? "
        cur = dbconnection.cursor()
        cur.execute(sql, [Date])
        result = cur.fetchone()
        block6 = input('your district:')
        block6 = block6.strip()
        AQHI_index(block6, dbconnection)
            
def selectAQHI_date(dbconnection):
    ALLDate(dbconnection)
    block7 = input("Please input your date:")
    block7 = block7.strip()
    AQHI_District(block7, dbconnection)

### Define insert function

In [65]:
# sentence 6: add
def UserADD(dbconnection):
    
    info1 = input('Please input your distincit user id,like this:(1314)')
    info2 = input('Please input the date, like this:(20200131)')
    info3 = input('Please input the district, like this:(Causeway Bay)')
    info4 = input('Please input the time, like this:(15)')
    info5 = input('Please input the Air Quality Index')
   
    with dbconnection:
        sql = "INSERT INTO AQHI VALUES(? ,?, ?, ?, ?)"
        cur = dbconnection.cursor()
        cur.execute(sql, (info1, info2, info3, info4,  info5))
        dbconnection.commit
        
    print('Add information successfully !')

### Define update function

In [66]:
# sentence 7: update
def UserUPDATE(dbconnection):
    
    ALLDate(dbconnection)
    UPinfo1 = input('Please choose which day you wanna update the information, format like this:(31/1/2020)')
    ALLDistrict(dbconnection)
    UPinfo2 = input('Please choose a district to update the information')
    UPinfo3 = input('Please input the time you want to update')
    UPinfo4 = input('Please input the Air Quality Index you want to update')
   
    with dbconnection:
        sql = "UPDATE AQHI SET Time = ? ,AQHI = ?  WHERE Date = ? and District = ?"
        cur = dbconnection.cursor()
        cur.execute(sql, (UPinfo3, UPinfo4, UPinfo1,  UPinfo2))
        dbconnection.commit
    
    print('Update successfully !')

### Define delete function

In [67]:
# sentence 8: Delete
def UserDelete(dbconnection):

    ALLDate(dbconnection)
    block11 = input('Please the date you want to delete:')
    block11 = block11.strip()
    print(block11)
    with dbconnection:
        
        sql = ('DELETE FROM AQHI WHERE DATE = ? ')
        cur = dbconnection.cursor()
        cur.execute(sql, [block11])
        dbconnection.commit() 
        print('Delet{',block11, '}successfully !')

## Part 3. Create the interface

In [68]:
def console():
    print('---------------------------The Hong Kong Air Quaility--------------------------------------------\n')
    print('· Query')
    print('1.The districts with good air quality in one day. \n2.Worst air quality day in the district. \n3.The average air quality index of a district')
    print('4.The value of main pollutants in a district on one day. \n5.The value of air quality index in a district on one day.')
    print('· Add, Update and Delete')
    print('6.Add information. \n7.Update the information. \n8.Delete information.')
    print('9.Exit. \n')
    print('--------------------------- User Operate --------------------------------------\n')


In [ ]:
while True:
    console()
    choice = input('Please input the number you want to choose')

    if choice =='1':
        selectday(dbconnection)

    elif choice =='2':
        selectdistrict(dbconnection)

    elif choice =='3':
        selectdistrictAvg(dbconnection)

    elif choice =='4':
        selectmain_date(dbconnection)

    elif choice =='5':
        selectAQHI_date(dbconnection)

    elif choice =='6':
        UserADD(dbconnection)

    elif choice =='7':
        UserUPDATE(dbconnection)

    elif choice =='8':
        UserDelete(dbconnection)
    
    elif choice =='9':
        dbconnection.close()
        break

    else:
        print('Please input the right number')
        continue

---------------------------The Hong Kong Air Quaility--------------------------------------------

· Query
1.The districts with good air quality in one day. 
2.Worst air quality day in the district. 
3.The average air quality index of a district
4.The value of main pollutants in a district on one day. 
5.The value of air quality index in a district on one day.
· Add, Update and Delete
6.Add information. 
7.Update the information. 
8.Delete information.
9.Exit. 

--------------------------- User Operate --------------------------------------

Please input the number you want to choose5
Please choose one day:
20200101
20200102
20200103
20200104
20200105
20200106
20200107
20200108
20200109
20200110
20200111
20200112
20200113
20200114
20200115
20200116
20200117
20200118
20200119
20200120
20200121
20200122
20200123
20200124
20200125
20200127
20200128
20200129
20200130
20200131
Please input your date:20200122
Please choose one district:
Causeway Bay
Central
Mong Kok
Tap Mun
Tseung Kwan 
Tsue